<a href="https://colab.research.google.com/github/Mun-Robbery/Mun-Robbery/blob/main/logistic_regresssion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [9]:
# 0) prepare data
bc = datasets.load_breast_cancer()
X , y = bc.data, bc.target

n_samples, n_features =  X.shape
print(n_samples, n_features)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

569 30


In [10]:
# scale
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test= torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [11]:
# 1) model
# f = wx + b, sigmoid at the end
class LogisticRegression(nn.Module):
  def __init__(self, n_inpup_features):
    super(LogisticRegression, self).__init__()
    self.linear = nn.Linear(n_inpup_features, 1)

  def forward(self, x):
    y_predicted = torch.sigmoid(self.linear(x))
    return y_predicted
model = LogisticRegression(n_features)

In [12]:
# 2) loss and optimiser
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [14]:
# 3) training loops
num_epoch = 100
for epoch in range(num_epoch):
  # forward pass
  y_predicted = model(X_train)
  loss = criterion(y_predicted, y_train)
  # backward pass
  loss.backward()
  # updates
  optimizer.step()
  # zero gradients
  optimizer.zero_grad()
  if (epoch + 1) % 10 == 0:
    print(f'epoch: {epoch + 1}, loss = {loss.item():.4f}')

with torch.no_grad():
  y_predicted = model(X_test)
  y_predicted_cls = y_predicted.round()
  acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
  print(f'accuracy = {acc:.4f}')


epoch: 10, loss = 0.5387
epoch: 20, loss = 0.4585
epoch: 30, loss = 0.4040
epoch: 40, loss = 0.3644
epoch: 50, loss = 0.3342
epoch: 60, loss = 0.3103
epoch: 70, loss = 0.2908
epoch: 80, loss = 0.2745
epoch: 90, loss = 0.2607
epoch: 100, loss = 0.2488
accuracy = 0.8947
